In [1]:
import sys
sys.path.append('../src')
from order_book import Book
from event import Event
from datetime import datetime
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio


# import lobster message file
cols = ['time', 'type', 'id', 'shares', 'price', 'direction']
data = pd.read_csv("../data/lobster/AAPL_2012-06-21_34200000_37800000_message_50.csv", names=cols)
# re-scale the price col
data.price = data.price/10000
# make sure data is during market hours
data = data[data['time']>= 9.5*60*60]
data = data[data['time']<= 16*60*60]
print(len(data))
data.head()

91997


,time,type,id,shares,price,direction
0,34200.004241,1,16113575,18,585.33,1
1,34200.004261,1,16113584,18,585.32,1
2,34200.004447,1,16113594,18,585.31,1
3,34200.025552,1,16120456,18,585.91,-1
4,34200.025580,1,16120480,18,585.92,-1


In [2]:
# create order book object
book = Book()
events = []

# iterate through event messages and read into book - test using first 2250 events
for i in range(2250):
    event = Event(data.loc[i])
    book.handleEvent(event, i)
    events.append(event)

In [3]:
# pull all the visible executions into a trades data frame
trades = book.getVisibleExecutions(split=False)
trades.head()

,Time,ID,Price,Shares,Direction
0,03:30:00.275016,5740544.0,585.74,40.0,-1.0
1,03:30:00.275016,3570647.0,585.75,25.0,-1.0
2,03:30:00.275057,3647217.0,585.73,1.0,1.0
3,03:30:00.275063,3647217.0,585.73,10.0,1.0
4,03:30:00.275072,3570647.0,585.75,25.0,-1.0


In [4]:
# get out lobster formatted book
# remove the first 2150 rows
# this is for demo purposes to create the gif below
formatted_book = book.formatBook(2150,1)
formatted_book.head()


,Ask_1,Ask_1_Vol,Ask_1_Ord,Bid_1,Bid_1_Vol,Bid_1_Ord,Time
0,585.46,200.0,2.0,585.28,7.0,1.0,03:31:26.037044
1,585.46,200.0,2.0,585.24,100.0,1.0,03:31:26.037044
2,585.46,200.0,2.0,585.27,9.0,1.0,03:31:26.037225
3,585.46,200.0,2.0,585.27,9.0,1.0,03:31:26.038864
4,585.46,200.0,2.0,585.27,23.0,2.0,03:31:26.038946


In [5]:
# take the bid and ask data
bid_ask_data = formatted_book
# sync the starting time stamp of the bid ask data with the trades data
trades = trades[trades['Time']>=bid_ask_data.iloc[0]['Time']]

In [6]:
# set up fig dict
my_fig_dict = {
    "data": [],
    "layout": {},
    "frames": []
}

In [7]:
# fill in most of layout
my_fig_dict["layout"]["xaxis"] = {"autorange": True, 'tickmode':'linear', 'dtick':'250ms', 'tickformat':'%S.%.3f'}
my_fig_dict["layout"]["yaxis"] = {"title": "AAPL Price"}
my_fig_dict["layout"]["hovermode"] = "closest"
my_fig_dict["layout"]["barmode"] = "stack"
my_fig_dict["layout"]["updatemenus"] = [
    {
        "buttons": [
            {"args": [None, {"frame": {"duration": 0, "redraw": False},"fromcurrent": True, "transition": {"duration": 0,"easing": "quadratic-in-out"}}],"label": "Play","method": "animate"},
            {"args": [[None], {"frame": {"duration": 0, "redraw": False},"mode": "immediate","transition": {"duration": 0}}],"label": "Pause","method": "animate"}
        ]
    }
]

my_sliders_dict = {"active": 0,"xanchor": "left","currentvalue": {"prefix": "Time:",},"pad": {"b": 0, "t": 120},"len": 0.9,"x": 0.05,"y": 0,"steps": []}

In [8]:
# create all the traces then add to the dictionary
# best bid line trace
best_bid_price_trace = dict(x=bid_ask_data.Time, y=bid_ask_data.Bid_1, mode='lines', line=dict(color='green'), name='best_bid')
my_fig_dict["data"].append(best_bid_price_trace)

# best ask line trace
best_ask_price_trace = dict(x=bid_ask_data.Time, y=bid_ask_data.Ask_1, mode='lines', line=dict(color='red'), name='best_ask')
my_fig_dict["data"].append(best_ask_price_trace)

# volume bar trace
traded_vol_trace = go.Bar(x=trades.Time, y=trades.Shares, marker=dict(color='blue'), name='traded volume')
my_fig_dict["data"].append(traded_vol_trace)

# trades scatter trace
trades_trace = dict(x=trades.Time, y=trades.Price, mode='markers', marker=dict(color='blue', size=np.log(trades.Shares)*5), name='Executions', text='Shares: ' + trades.Shares.astype(str))
my_fig_dict["data"].append(trades_trace)

In [9]:
# itterate through each time step and create a frame 
# we can layer the frames to scroll trough the plot

for time_step in bid_ask_data.Time:
    # set up current frame
    my_frame ={'data': [], 'name': str(time_step)}

    # trim each data frame such that it only contains data from the start of the time series to the current time
    time_step_data = bid_ask_data[bid_ask_data['Time'] <= time_step]
    time_step_trades_data = trades[trades['Time'] <= time_step]
    
    # best bid line trace
    best_bid_price_time_trace = dict(x=time_step_data.Time, y=time_step_data.Bid_1, mode='lines', line=dict(color='green'), name='best_bid')
    my_frame["data"].append(best_bid_price_time_trace)

    # best ask line trace
    best_ask_price_time_trace = dict(x=time_step_data.Time, y=time_step_data.Ask_1, mode='lines', line=dict(color='red'), name='best_ask')
    my_frame["data"].append(best_ask_price_time_trace)

    # volume bar trace
    traded_vol_time_trace = go.Bar(x=time_step_trades_data.Time, y=time_step_trades_data.Shares, marker=dict(color='blue'), name='traded_vol')
    my_frame["data"].append(traded_vol_time_trace)

    # trades scarrter trace
    trades_time_trace = dict(x=time_step_trades_data.Time, y=time_step_trades_data.Price, mode='markers', marker=dict(color='blue', size=np.log(trades.Shares)*5), name='Executions', text='Shares: ' + trades.Shares.astype(str))
    my_frame["data"].append(trades_time_trace)

    # append the traces to the frames dictionary
    # update the slider
    my_fig_dict["frames"].append(my_frame)
    my_slider_step = {"args": [[time_step],{"frame": {"duration": 0, "redraw": False},"mode": "immediate","transition": {"duration": 0}}],"label": str(time_step),"method": "animate"}
    my_sliders_dict["steps"].append(my_slider_step)

In [10]:
# add the full slider to the layout
my_fig_dict["layout"]["sliders"] = [my_sliders_dict]

# plot the data and create an upper and lower sub plot
fig = go.Figure(my_fig_dict).set_subplots(2, 1, subplot_titles=('Top of Book Price', 'Volume Traded'), vertical_spacing=0.05, row_heights=[0.6, 0.4], shared_xaxes=True)
# set x and y for volume bar chart
fig.data[2].xaxis = "x2"
fig.data[2].yaxis = "y2"
fig.update_layout(yaxis2_title='Shares', xaxis2_title='Time')
fig.update_layout(width=1440, height=722, margin=dict(l=50, r=50, b=75, t=50), yaxis2_range=[0,250])
fig.show()
#fig.write_html("../data/html/AAPL_L1_Data.html")


In [11]:
# write to gif 
import io
import PIL
frames = []
for s, fr in enumerate(fig.frames):
    # set main traces to appropriate traces within plotly frame
    fig.update(data=fr.data)
    # move slider to correct place
    fig.layout.sliders[0].update(active=s)
    # generate image of current state
    frames.append(PIL.Image.open(io.BytesIO(fig.to_image(format="png"))))

frames[0].save(
        "../data/gif/bid_ask_vol.gif",
        save_all=True,
        append_images=frames[1:],
        optimize=True,
        duration=250,
        loop=0,
    )

!['bid_ask_vol'](../data/images/bid_ask_vol.png)

On the GIF below, the stuttering is due to sequential events from the message file that have the same time stamp
We can see this when we zoom into the volume bars on the lower subplot 

!['bid_ask_vol_stacked'](../data/images/bid_ask_vol_stacked_bar.png)

!['bid_ask_vol.gif'](../data/gif/bid_ask_vol.gif)